In [ ]:
!pip install rasterio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 46.8 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
from scipy.spatial import KDTree
from PIL import Image
import json
import os
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.transform import Affine
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def hex_to_rgb(hex_color):
    """Convierte un color HEX a un array RGB."""
    hex_color = hex_color.lstrip('#')
    return np.array([int(hex_color[i:i+2], 16) for i in (0, 2, 4)])

def generate_sub_regions(bbox, scale_h, scale_w):
    """
    Genera sub-bboxes en formato [min_lon, min_lat, max_lon, max_lat].
    """

    min_lon, min_lat, max_lon, max_lat = bbox

    width = max_lon - min_lon
    height = max_lat - min_lat

    sub_regions = []
    sub_width = width / scale_w
    sub_height = height / scale_h

    for i in range(scale_h):
        for j in range(scale_w):
            new_min_lon = min_lon + j * sub_width
            new_min_lat = min_lat + i * sub_height
            new_max_lon = new_min_lon + sub_width
            new_max_lat = new_min_lat + sub_height
            region = RegionMap(1, 1, [new_min_lon, new_min_lat, new_max_lon, new_max_lat])
            sub_regions.append(region)
    return sub_regions

def bbox_to_polygon(bbox):
    return Polygon([
        [bbox[0], bbox[1]],  # min
        [bbox[2], bbox[1]],  # max_lon,
        [bbox[2], bbox[3]],  # max_lat
        [bbox[0], bbox[3]],  # min_lat
        [bbox[0], bbox[1]]   # Cierre
    ])

def get_pixel_scale(bbox, image_width, image_height):
    min_lon, min_lat, max_lon, max_lat = bbox
    width = max_lon - min_lon
    height = max_lat - min_lat
    return width / image_width, height / image_height

def plot_matrix(matrix, range=None, colors=None):
    cmap = 'gray'
    if colors:
        cmap = mcolors.LinearSegmentedColormap.from_list("alerta", colors)
    if range is not None:
        vmin, vmax = range
        plt.imshow(matrix, cmap=cmap, vmin=vmin, vmax=vmax)
    else:
        plt.imshow(matrix, cmap=cmap)
    plt.colorbar()
    plt.show()


def resize_nearest_neighbor_pil(matrix, new_width, new_height):
    image = Image.fromarray(matrix)  # Convierte la matriz en imagen
    resized_image = image.resize((new_width, new_height), Image.NEAREST)  # Escalado con vecino más cercano
    return np.array(resized_image)  # Devuelve como matriz

# class layerRegion():
#     def __init__(self):
#         pass

class ScaleLayer:
    def __init__(self, file_path=None, values=None):
        if file_path is not None:
            obj = json.load(open(file_path))

        else:
            obj = values

        self.values = [o.get('value', None) for o in obj['values']]
        self.colors = [o.get('color', None) for o in obj['values']]
        self.labels = [o.get('label', None) for o in obj['values']]

    def image_to_matrix(self, image_array):
        height, width, _ = image_array.shape
        color_values = [hex_to_rgb(c) for c in self.colors]
        tree = KDTree(np.array(color_values))
        matrix = np.zeros((height, width))
        for i in range(height):
            for j in range(width):
                pixel_color = image_array[i, j]  # Mantener en formato 0-255
                _, idx = tree.query(pixel_color)
                matrix[i, j] = self.values[idx]
        return matrix

class Layer:
    def __init__(self, scale = None, pixel_resolution=None, relevance_factor=1):
        self.scale = scale
        self.matrix = None
        self.matrix_file = None
        self.pixel_resolution = pixel_resolution
        self.bbox = None
        self.relevance_factor = relevance_factor

    def geosjon_to_matrix(self, gdf):
        bounds = gdf.total_bounds
        pixel_resolution = self.pixel_resolution
        value_field = 'value'
        width = int((bounds[2] - bounds[0]) / pixel_resolution)
        height = int((bounds[3] - bounds[1]) / pixel_resolution)
        transform = from_origin(bounds[0], bounds[3], pixel_resolution, pixel_resolution)
        shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[value_field])]
        matrix = rasterize(
            shapes,
            out_shape=(height, width),
            transform=transform,
            fill=0,
            dtype=np.float32
        )

        return matrix

    def load_matrix(self, func):
        if self.matrix_file is not None and os.path.exists(self.matrix_file):
            matrix_file_loaded = np.load(self.matrix_file)
            self.matrix = matrix_file_loaded['matrix']
            self.pixel_resolution = matrix_file_loaded['pixel_resolution']
            self.bbox = matrix_file_loaded['bbox']
        else:
            func()
            self.save_matrix()

    def save_matrix(self):
        np.savez_compressed(
            self.matrix_file,
            matrix=self.matrix,
            pixel_resolution=self.pixel_resolution,
            bbox=self.bbox
        )

    def matrix_by_bbox(self, bbox, out_resolution):
        q_xmin, q_ymin, q_xmax, q_ymax = bbox
        resolution = self.pixel_resolution
        transform = Affine.translation(bbox[0], bbox[3]) * Affine.scale(resolution, -resolution)

        with rasterio.io.MemoryFile().open(
            driver='GTiff',
            height=self.matrix.shape[0],
            width=self.matrix.shape[1],
            count=1,
            dtype=self.matrix.dtype,
            transform=transform
        ) as dataset:
            dataset.write(self.matrix, 1)
            window = dataset.window(*bbox)
            matrix_window = dataset.read(1, window=window, boundless=True, fill_value=np.nan)
            if out_resolution == resolution:
                return matrix_window
            output_width = int((q_xmax - q_xmin) / out_resolution)
            output_height = int((q_ymax - q_ymin) / out_resolution)
            return resize_nearest_neighbor_pil(matrix_window, output_width, output_height)


    def pre_process(self, region_map):
        pass

    def process(self, region_map):
        pass

    def post_process(self, region_map):
        pass


class LayerRegion():
    def __init__(self, matrix=None, layer=None, weighted_sum=None):
        self.matrix = matrix
        self.layer = layer
        self.weighted_sum = weighted_sum

    def merge(self, regionLayer):
        if self.weighted_sum == None or self.weighted_sum == 0:
            return self

        if regionLayer.layer == None:
            return self

        if self.matrix is None:
            regionLayer.weighted_sum = self.weighted_sum
            regionLayer.matrix = regionLayer.matrix * regionLayer.layer.relevance_factor / self.weighted_sum
            return regionLayer

        self.matrix = self.matrix + regionLayer.matrix * regionLayer.layer.relevance_factor / self.weighted_sum
        return self

class ElevationLayer(Layer):
    def __init__(self, scale=None, pixel_resolution=None):
        super().__init__(scale, pixel_resolution)
        self.matrix_file = 'cusco-elevation.npz'

    def fetch_map(self):
        image = Image.open("cusco-elevation.jpg")
        image_array = np.array(image)
        self.bbox = [-13.57388, -72.02083, -13.48333, -71.85583]
        height, width, _ = image_array.shape
        lat_min, lon_min, lat_max, lon_max = self.bbox
        res_lat = round((lat_max - lat_min) / height, 5)
        res_lon = round((lon_max - lon_min) / width, 5)
        self.pixel_resolution = min(res_lat, res_lon)
        self.matrix = self.scale.image_to_matrix(image_array)

    def pre_process(self, region_map):
        print("PRE_PROCESS ElevationLayer")
        self.load_matrix(self.fetch_map)

    def process(self, region_map):
        print("PROCESS ElevationLayer")
        sub_matrix = self.matrix_by_bbox(region_map.bbox, region_map.pixel_resolution)
        layer_region = LayerRegion(sub_matrix, layer=self)
        return layer_region

    def post_process(self, region_map):
        pass

class ElevationRiskLayer(ElevationLayer):
    def __init__(self, scale=None, pixel_resolution=None, relevance_factor=1):
        super().__init__(scale, pixel_resolution)
        self.relevance_factor = relevance_factor

    def to_risk_map(self):
        matrix = self.matrix
        threshold = 3500
        min_val = np.min(matrix)
        result = 1 - np.where(
            matrix > threshold,
            1,
            (matrix - min_val) / (threshold - min_val)
        )
        self.matrix = result

    def pre_process(self, region_map):
        print("PRE_PROCESS ElevationLayer")
        self.load_matrix(self.fetch_map)
        self.to_risk_map()

class RiverLayer(Layer):
    def __init__(self, scale=None, pixel_resolution=None, relevance_factor=1):
        super().__init__(scale, pixel_resolution)
        self.matrix_file = 'cusco-rivers.npz'
        self.relevance_factor= relevance_factor

    def fetch_map(self):
        rivers_gdf = gpd.read_file('cusco_river.json')
        self.bbox = rivers_gdf.total_bounds
        rivers_gdf['value'] = 0.9
        self.pixel_resolution = 0.0003
        self.matrix = self.geosjon_to_matrix(rivers_gdf)
        self.bbox = rivers_gdf.total_bounds

    def pre_process(self, region_map):
        self.load_matrix(self.fetch_map)
        # plot_matrix(self.matrix, (0, 1))

    def process(self, region_map):
        print("PROCESS RiverLayer")
        sub_matrix = self.matrix_by_bbox(region_map.bbox, region_map.pixel_resolution)
        layer_region = LayerRegion(sub_matrix, layer=self)
        return layer_region

    def post_process(self, region_map):
        pass


class RegionMap:
    def __init__(self, scale_h, scale_w, bbox=None, geojson=None, pixel_resolution=0.0006):
        if bbox is None and geojson is None:
            raise ValueError("NO_BBOX_OR_GEOJSON")
        if geojson is not None:
            gdf = gpd.read_file(geojson)
            bbox = gdf.total_bounds
            self.gdf = gdf

        self.bbox = bbox  # Format: [min_lon, min_lat, max_lon, max_lat]
        self.scale_h = scale_h
        self.scale_w = scale_w
        self.pixel_resolution = pixel_resolution
        self.regions = []
        self.polygons = []
        self.layers = []
        self.deep_layers = []
        if scale_h * scale_w > 1:
            self.regions = generate_sub_regions(bbox, scale_h, scale_w)

    def bbox_to_gpd(self):
        bbox = self.bbox
        polygon = bbox_to_polygon(bbox)
        gdf = gpd.GeoDataFrame(geometry=[polygon], crs="EPSG:4326")
        return gdf

    def regions_to_gpd(self):
        regions = []
        for region in self.regions:
            bbox = region.bbox
            polygon = bbox_to_polygon(bbox)
            regions.append(polygon)

        gdf = gpd.GeoDataFrame(geometry=regions, crs="EPSG:4326")
        return gdf

    def add_layer(self, layer, deep=False):
        if deep == False: self.layers.append(layer)
        else: self.deep_layers.append(layer)

    def set_layers(self, layers, deep=False):
        if deep == False: self.layers = layers
        else: self.deep_layers = layers

    def base_process_layers(self):
        weighted_sum = 0
        for layer in self.layers:
            layer.pre_process(self)
            if layer.relevance_factor is not None:
                weighted_sum += layer.relevance_factor
        layer_region = LayerRegion(weighted_sum=weighted_sum)
        for layer in self.layers:
            layer_region = layer_region.merge(layer.process(self))
        for layer in self.layers:
            layer.post_process(self)
        return layer_region

    def process_layers(self, scale=None):
        layer_region = self.base_process_layers()
        if len(self.deep_layers) > 0:
            for region in self.regions:
                region.set_layers(self.deep_layers, deep=False)
                layer_region = layer_region.merge(region.base_process_layers())

        plot_matrix(layer_region.matrix, (0, 1), scale.colors)

    def add_polygon(self, polygon):
        self.polygons.append(polygon)

In [ ]:
!pip install geemap --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.3 MB/s eta 0:00:00


In [ ]:
import ee
ee.Authenticate()
EE_PROJECT_ID = 'ee-narosqui'
ee.Initialize(project=EE_PROJECT_ID)
import math
import geemap

def calculate_bbox_dimensions(region):
    coords = region.coordinates().getInfo()[0]
    x_distance = ee.Geometry.LineString([coords[0], coords[1]]).length().getInfo()  # Ancho en metros
    y_distance = ee.Geometry.LineString([coords[1], coords[2]]).length().getInfo()  # Altura en metros
    return x_distance, y_distance

def show_image(image, bands, bbox, title='', path=''):
    # Recortar la imagen al área del bbox
    region = ee.Geometry.Rectangle(bbox)

    # Parámetros
    resolution = 10  # metros por píxel

    x_distance, y_distance = calculate_bbox_dimensions(region)
    x_pixels = math.ceil(x_distance / resolution)
    y_pixels = math.ceil(y_distance / resolution)

    print(f"Resolución: {resolution} m/píxel — Tamaño estimado: {x_pixels} x {y_pixels} píxeles")

    # Convertir imagen a NumPy
    rgb_array = geemap.ee_to_numpy(image.select(bands), region=region, bands=bands, scale=resolution)
    rgb_array = rgb_array.astype('float32')
    for i in range(rgb_array.shape[2]):
        band = rgb_array[:, :, i]
        rgb_array[:, :, i] = (band - band.min()) / (band.max() - band.min() + 1e-6)

    # Plot
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(rgb_array)
    # plt.title(title)

    plt.axis('off')
    plt.savefig(path, bbox_inches='tight', pad_inches=0, transparent=False)
    plt.close(fig)
    # plt.show()

def plot_rgb(image, bbox, path):
    show_image(image, ['B4', 'B3', 'B2'], bbox, title='RGB B4-B3-B2', path=path)

def plot_rgb_water(image, bbox, path):
    show_image(image, ['B11', 'B8', 'B4'], bbox, title='RGB B11-B8-B4 (Agua)',path=path)

def plot_ndwi(image, bbox, path):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    show_image(ndwi, ['NDWI'], bbox, title='NDWI',path=path)

def plot_mndwi(image, bbox, path):
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    show_image(mndwi, ['MNDWI'], bbox, title='MNDWI',path=path)

def plot_awei(image, bbox, path):
    b3 = image.select('B3')
    b8 = image.select('B8')
    b11 = image.select('B11')
    b12 = image.select('B12')

    awei = b3.subtract(b8).multiply(4) \
             .subtract(b11.multiply(0.25)) \
             .subtract(b12.multiply(2.75)) \
             .rename('AWEI')

    show_image(awei, ['AWEI'], bbox, title='AWEI')

def get_images_by_bbox(idx, bbox, start_date, end_date, cloud=5):
  s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(ee.Geometry.Rectangle(bbox)) \
    .filter(ee.Filter.lessThan('CLOUDY_PIXEL_PERCENTAGE', cloud)) \
    .map(lambda img: img.clip(ee.Geometry.Rectangle(bbox)))

  os.makedirs('images_rgb_water', exist_ok=True)
  os.makedirs(f'images_rgb_water/{idx}', exist_ok=True)

  os.makedirs('images_rgb', exist_ok=True)
  os.makedirs(f'images_rgb/{idx}', exist_ok=True)

  # os.makedirs('images_ndwi', exist_ok=True)
  # os.makedirs(f'images_ndwi/{idx}', exist_ok=True)


  list_images = s2.toList(s2.size())
  for i in range(list_images.size().getInfo()):
    image = ee.Image(list_images.get(i))
    date_acquisition = ee.Date(image.get('system:time_start'))
    date = date_acquisition.format('YYYY-MM-dd').getInfo()
    cloud_coverage = image.get('CLOUD_COVERAGE_ASSESSMENT').getInfo()
    path_rgb_water = f"images_rgb_water/{idx}/{date}.jpg"
    path_rgb = f"images_rgb/{idx}/{date}.jpg"
    # path_ndwi = f"images_ndwi/{idx}/{date}.jpg"

    plot_rgb_water(image,bbox,path_rgb_water)
    plot_rgb(image,bbox, path_rgb)
    # plot_ndwi(image,bbox, path_ndwi)

In [ ]:
sat_region_map = RegionMap(scale_h=1, scale_w=1, geojson='blanca_grid_5km_select_select.json')


In [ ]:
bboxes = sat_region_map.gdf.geometry.bounds

start_time = '2016-01-01'
end_time = '2025-04-18'


# for i in range(136, 146):
for i in [39,60,71,94,110,127,136]:
    bbox = bboxes.iloc[i].tolist()
    print(i, bbox)
    get_images_by_bbox(i, bbox, start_time, end_time)

In [ ]:
#delete folder
!rm -rf images_rgb/
!rm -rf images_rgb_water/
!rm images_rbg.zip
!rm images_rbg_water.zip

rm: cannot remove 'images_rbg.zip': No such file or directory
rm: cannot remove 'images_rbg_water.zip': No such file or directory


In [ ]:
#zip folder
!zip -r images_rbg.zip images_rgb

!zip -r images_rbg_water.zip images_rgb_water

# !zip -r images_ndwi.zip images_ndwi



In [ ]:
# clean memor

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import flood
from skimage.io import imread

# Cargar imagen
image_path = "images/2016-05-29.png"  # Actualiza con la ruta de tu imagen
image = imread(image_path)

# Lista de puntos dentro del lago (ej: [(y1, x1), (y2, x2), ...])
puntos_lago = [(340,73), (321,164), (322,220)]  # Cambia por tus puntos reales
plt.figure(figsize=(8, 8))
plt.imshow(image)
ys, xs = zip(*puntos_lago)  # separa coordenadas
plt.scatter(xs, ys, color='red', s=30, marker='x', label='Puntos del lago')
plt.title("Imagen con puntos del lago")
plt.axis("on")
plt.legend()
plt.show()
from skimage.color import rgb2gray

# Convertir a escala de grises para segmentación basada en intensidad
gray = rgb2gray(image[..., :3])

# Crear una máscara vacía para acumular resultados de flood
mask_total = np.zeros_like(gray, dtype=bool)

# Aplicar flood desde cada punto inicial
for punto in puntos_lago:
    y, x = punto
    mask = flood(gray, seed_point=(y, x), tolerance=0.05)
    mask_total = np.logical_or(mask_total, mask)

# Visualizar máscara
plt.imshow(mask_total, cmap='gray')
plt.title("Segmentación del lago")
plt.axis("off")
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread

# Cargar imagen y quitar canal alfa si es necesario
image_rgba = imread("images/2016-05-29.png")
image = image_rgba[..., :3]

# Convertir a escala de grises
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Umbral para detectar posibles zonas de lago
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Área segura de fondo (dilatado)
kernel = np.ones((6, 6), np.uint8)
sure_bg = cv2.dilate(thresh, kernel, iterations=3)

# Crear marcador vacío
markers = np.zeros(gray.shape, dtype=np.int32)

# Agregar fondo como marcador 1
markers[sure_bg == 255] = 1

# Tus puntos de lago como marcadores únicos (2, 3, 4...)
puntos_lago = [(340, 73), (321, 164), (322, 220)]
for idx, (y, x) in enumerate(puntos_lago):
    cv2.circle(markers, (x, y), radius=5, color=(idx + 2), thickness=-1)  # marcador: 2, 3, 4

# Aplicar watershed
image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
markers = cv2.watershed(image_bgr, markers)

# Dibujar los contornos segmentados
output = image_bgr.copy()
output[markers == -1] = [0, 0, 255]  # Bordes en rojo

# Opcional: Colorear cada lago diferente
colored = np.zeros_like(image_bgr)
for idx in range(2, len(puntos_lago) + 2):
    colored[markers == idx] = np.random.randint(0, 255, size=3)

# Mostrar resultados
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.title("Contornos con Watershed")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(colored, cv2.COLOR_BGR2RGB))
plt.title("Segmentación coloreada")
plt.axis("off")

plt.show()


In [ ]:
import numpy as np
import cv2
from skimage.io import imread
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

# Ruta a la imagen
image = imread("images/2016-05-29.png")[..., :3]
h, w, _ = image.shape

# Features por píxel: [R, G, B, x, y]
X = []
for y in range(h):
    for x in range(w):
        pixel = image[y, x]
        X.append([pixel[0], pixel[1], pixel[2], x, y])
X = np.array(X)

# Puntos de interés etiquetados como lago
puntos_lago = [(340, 73), (321, 164), (322, 220)]
y_labels = np.zeros((h, w), dtype=np.uint8)

for y, x in puntos_lago:
    cv2.circle(y_labels, (x, y), radius=5, color=1, thickness=-1)

# Puntos de entrenamiento
train_idxs = np.column_stack(np.where(y_labels > 0))
n_no_lago = len(train_idxs) * 3
no_lago_idxs = np.column_stack(np.where(y_labels == 0))
no_lago_sample = no_lago_idxs[np.random.choice(len(no_lago_idxs), n_no_lago, replace=False)]

train_coords = np.vstack((train_idxs, no_lago_sample))
train_labels = np.array([1] * len(train_idxs) + [0] * len(no_lago_sample))

# Features para entrenamiento
train_features = np.array([X[y * w + x] for y, x in train_coords])

# Clasificador
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(train_features, train_labels)

# Predicción para toda la imagen
y_pred = clf.predict(X).reshape(h, w)

# Mostrar resultados
plt.figure(figsize=(16, 6))

plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Imagen original")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(y_pred, cmap="Blues")
plt.title("Segmentación con Random Forest")
plt.axis("off")

plt.show()
